---
<h1 style="text-align: center;">Lista 5 - Exercícios Redes Neurais:</h1>
<h2 style="text-align: center;">Prof. Dr. Jodavid Ferreira</h2>

*Discente:*

    *   Gabriel D'assumpção de Carvalho

*Data:* 05/08/2024

---
---

In [14]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import models
from tensorflow.keras.layers import Dense, Dropout, Conv2D, Input, Flatten, MaxPooling2D
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

import matplotlib.pyplot as plt

1. **Crie uma rede neural totalmente conectada com duas camadas ocultas, cada uma com 128 neurônios, utilizando a biblioteca keras. Utilize a função de ativação ReLU e a função de perda categorical_crossentropy para treinar o modelo no conjunto de dados abaixo:**

* Base de dados: https://www.kaggle.com/datasets/abdelazizsami/wine-quality

In [2]:
wine_white = pd.read_csv(
    "https://raw.githubusercontent.com/gabrieldadcarvalho/neural_network/main/listas/05/data/winequality-white.csv",
    sep=";",
)
wine_red = pd.read_csv(
    "https://raw.githubusercontent.com/gabrieldadcarvalho/neural_network/main/listas/05/data/winequality-red.csv",
    sep=";",
)

In [3]:
wine_white["color"] = 0
wine_red["color"] = 1

In [4]:
df = pd.concat([wine_white, wine_red], ignore_index=True)
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,color
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6,0
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6,0
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6,0
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6,0
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6,0


In [5]:
df.tail()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,color
6492,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5,1
6493,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,6,1
6494,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6,1
6495,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5,1
6496,6.0,0.310,0.47,3.6,0.067,18.0,42.0,0.99549,3.39,0.66,11.0,6,1


In [6]:
X = df.drop(columns="quality")
X

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,color
0,7.0,0.270,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,0
1,6.3,0.300,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,0
2,8.1,0.280,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,0
3,7.2,0.230,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,0
4,7.2,0.230,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,0
...,...,...,...,...,...,...,...,...,...,...,...,...
6492,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,1
6493,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,1
6494,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,1
6495,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,1


In [7]:
y = df["quality"]
y

0       6
1       6
2       6
3       6
4       6
       ..
6492    5
6493    6
6494    6
6495    5
6496    6
Name: quality, Length: 6497, dtype: int64

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [9]:
def nn():
    model = models.Sequential()
    model.add(Dense(128, activation="relu", input_shape=(X.shape[1],)))
    model.add(Dense(128, activation="relu"))
    model.add(Dense(10, activation="softmax"))
    model.compile(
        optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
    )
    return model

In [10]:
model = nn()
model.summary()

/home/gabrieldadcarvalho/anaconda3/envs/am/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1725710844.342142    5703 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-09-07 09:07:24.647356: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2343] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required librarie

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 128)            │         1,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 19,466 (76.04 KB)

 Trainable params: 19,466 (76.04 KB)

 Non-trainable params: 0 (0.00 B)

In [11]:
history = model.fit(
    x=X_train, y=y_train, batch_size=32, epochs=100, validation_split=0.3
)

Epoch 1/100
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.3783 - loss: 5.3453 - val_accuracy: 0.4615 - val_loss: 1.2647
Epoch 2/100
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4085 - loss: 1.3216 - val_accuracy: 0.4256 - val_loss: 1.2675
Epoch 3/100
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4472 - loss: 1.2763 - val_accuracy: 0.4520 - val_loss: 1.2333
Epoch 4/100
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4386 - loss: 1.2639 - val_accuracy: 0.4161 - val_loss: 1.2601
Epoch 5/100
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4380 - loss: 1.2986 - val_accuracy: 0.3656 - val_loss: 1.4169
Epoch 6/100
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4448 - loss: 1.2560 - val_accuracy: 0.4381 - val_loss: 1.2127
Epoch 7/100
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4614 - loss: 1.2520 - val_accuracy: 0.4278 - val_loss: 1.2173
Epoch 8/100
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4489 - loss: 1.2522 - val_accu

2. **Modifique o modelo anterior para incluir uma camada de Dropout após cada camada oculta, utilizando uma taxa de dropout de 0.5. Explique como o Dropout ajuda a prevenir o overfitting no modelo.**

In [15]:
def nn_drop():
    modelo = models.Sequential()
    modelo.add(Dense(128, activation="relu", input_shape=(X_train.shape[1],)))
    modelo.add(Dropout(0.5))
    modelo.add(Dense(128, activation="relu"))
    modelo.add(Dropout(0.5))
    modelo.add(Dense(10, activation="softmax"))
    modelo.compile(
        optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
    )
    return modelo

In [16]:
modelo = nn_drop()

In [17]:
modelo.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_4 (Dense)                 │ (None, 128)            │         1,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 19,466 (76.04 KB)

 Trainable params: 19,466 (76.04 KB)

 Non-trainable params: 0 (0.00 B)

In [18]:
hisotry_dropout = modelo.fit(
    X_train, y_train, batch_size=32, epochs=100, validation_split=0.3
)

Epoch 1/100
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.2845 - loss: 11.3594 - val_accuracy: 0.3597 - val_loss: 1.5092
Epoch 2/100
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3265 - loss: 2.7991 - val_accuracy: 0.3487 - val_loss: 1.3304
Epoch 3/100
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3539 - loss: 1.8299 - val_accuracy: 0.4220 - val_loss: 1.3086
Epoch 4/100
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3785 - loss: 1.5527 - val_accuracy: 0.4220 - val_loss: 1.3172
Epoch 5/100
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4116 - loss: 1.5066 - val_accuracy: 0.4220 - val_loss: 1.2601
Epoch 6/100
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4235 - loss: 1.4017 - val_accuracy: 0.4212 - val_loss: 1.2642
Epoch 7/100
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4240 - loss: 1.4080 - val_accuracy: 0.4220 - val_loss: 1.2512
Epoch 8/100
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4353 - loss: 1.3543 - val_acc

**O dropout ajuda o modelo a previnir overfiting, modificando a estruturada da rede a cada época. Ele desativa aleatoriamente alguns neurônios, o que impede que certos pesos se ajustem excessivamente, forçando o modelo a ser mais robusto e generalizando melhor**

3. **Treine e avalie o desempenho do modelo original (sem Dropout) e do modelo com Dropout no conjunto de dados MNIST utilizando valores de 0.2, 0.5, e 0.8 para o Dropout. Compare as curvas de perda e acurácia durante o treinamento e comente sobre os resultados. Avalie também o impacto dessas variações no desempenho do modelo e na capacidade de generalização.**


In [ ]:
# Carregando conjunto de dados
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [ ]:
X_train = X_train.astype("float32") / 255.0
X_test = X_test.astype("float32") / 255.0

In [ ]:
X_train.shape

(60000, 28, 28)

In [ ]:
X_train = np.expand_dims(X_train, axis=-1)
X_test = np.expand_dims(X_test, axis=-1)

In [ ]:
X_train.shape

(60000, 28, 28, 1)

In [ ]:
filtro_h = np.array([[-1, 0, 1], [-1, 0, 1], [-1, 0, 1]], dtype=np.float32)

In [ ]:
filtro_h.shape

(3, 3)

In [ ]:
"""
# Dimensões:
-> 3,3: O tamanho do kernel (3x3).
-> 1 (terceira dimensão): Escala de cor da imagem que nesse caso é cinza.
-> 1 (quarta dimensão): Quantidade de filtro que esta sendo aplicado.
"""

filtro_h = filtro_h.reshape((3, 3, 1, 1))

In [ ]:
filtro_h.shape[:2]

(3, 3)

In [ ]:
len(filtro_h)

3

In [ ]:
def cnn(X_train, n_f, filtro_h):
    model = models.Sequential()
    model.add(
        Conv2D(
            filters=n_f,  # Defina explicitamente o número de filtros, ou ajuste conforme necessário
            kernel_size=filtro_h.shape[:2],  # Pega o tamanho correto do kernel
            padding="same",  # Mantém o tamanho da saída igual à entrada
            use_bias=False,  # Não utiliza viés
            kernel_initializer=tf.constant_initializer(
                filtro_h
            ),  # Inicializa com o filtro fornecido
            trainable=False,  # O filtro não será treinado
            input_shape=X_train.shape[1:],  # Corrigido o formato da entrada
        )
)
    return model

In [ ]:
model = cnn(X_train, 1, filtro_h)

/home/gabrieldadcarvalho/anaconda3/envs/am/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1725544867.462503   25190 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-09-05 11:01:07.465915: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2343] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the req

In [71]:
print(history.history.keys())

dict_keys(['accuracy', 'loss', 'val_accuracy', 'val_loss'])


In [ ]:
model.add(MaxPooling2D(pool_size=(2, 21)))

In [ ]:
model.add(Flatten())

In [ ]:
model.add(Dense(15, activation="relu"))

In [ ]:
n_categoria = len(np.unique(y_train))

In [ ]:
model.add(Dense(n_categoria, activation="softmax"))

In [ ]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 28, 28, 1)      │             9 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 14, 1, 1)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 14)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 15)             │           225 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │           160 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 394 (1.54 KB)

 Trainable params: 385 (1.50 KB)

 Non-trainable params: 9 (36.00 B)

In [ ]:
model.compile(
    optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)

In [ ]:
history = model.fit(
    X_train,
    y_train,
    batch_size=32,
    epochs=20,
    validation_split=0.3,
)

Epoch 1/20
1313/1313 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.3295 - loss: 1.9650 - val_accuracy: 0.5408 - val_loss: 1.3616
Epoch 2/20
1313/1313 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.5528 - loss: 1.3214 - val_accuracy: 0.5856 - val_loss: 1.2139
Epoch 3/20
1313/1313 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.5980 - loss: 1.2028 - val_accuracy: 0.6016 - val_loss: 1.1600
Epoch 4/20
1313/1313 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.6152 - loss: 1.1499 - val_accuracy: 0.6184 - val_loss: 1.1245
Epoch 5/20
1313/1313 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.6226 - loss: 1.1246 - val_accuracy: 0.6263 - val_loss: 1.0977
Epoch 6/20
1313/1313 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.6342 - loss: 1.0999 - val_accuracy: 0.6298 - val_loss: 1.0825
Epoch 7/20
1313/1313 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.6366 - loss: 1.0782 - val_accuracy: 0.6336 - val_loss: 1.0707
Epoch 8/20
1313/1313 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.6370 - loss: 1.0687 - 

In [ ]:
import tensorflow as tf

# Verificar se a GPU está disponível
print("Num GPUs Available: ", len(tf.config.list_physical_devices("GPU")))

# Exibir detalhes sobre a GPU
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

2024-09-05 11:40:57.102700: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-05 11:40:57.316541: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-05 11:40:57.364903: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-05 11:40:57.820158: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-05 11:41:01.331933: W tensorflow/compiler/tf2

Num GPUs Available:  0
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 4960001861402292207
xla_global_id: -1
]


I0000 00:00:1725547265.819618    4975 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-09-05 11:41:06.044869: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2343] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
I0000 00:00:1725547266.045891    4975 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-0